https://hd.stheadline.com/news/realtime/fin/2391104/%E5%8D%B3%E6%99%82-%E9%87%91%E8%9E%8D-%E6%B8%A3%E6%89%93%E4%BF%A1%E7%94%A8%E5%8D%A1%E7%88%86%E5%A4%A7%E8%A6%8F%E6%A8%A1%E7%9B%9C%E7%94%A8-%E5%AE%A2%E6%9C%8D%E5%94%94%E9%80%9A-%E6%89%8B%E6%A9%9F%E6%AD%BBApp-%E6%B8%A3%E6%89%93%E6%99%9A%E4%B8%8A%E6%94%B9%E5%AE%89%E6%8E%92-%E5%B0%87%E4%B8%BB%E5%8B%95%E9%80%80%E6%AC%BE-%E7%84%A1%E8%AD%89%E6%93%9A%E8%B3%87%E6%96%99%E5%A4%96%E6%B4%A9

In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
import sys
import time
import requests
from itertools import cycle
from datetime import datetime
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import NoSuchWindowException
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.wait import WebDriverWait
import undetected_chromedriver as uc
from datetime import timedelta, date, datetime

In [2]:
#functions
def click_thread(idx):
    #click thread
    auto_xpath = f'//*[@id="leftPanel"]/div[2]/div[{idx + 1}]/div/div[2]/a[1]'
    try:
        driver.find_element(By.XPATH, auto_xpath).click()
        #print(idx)
    except:
        #print("skip")
        pass
    time.sleep(0.3)

In [3]:
#Open pandas
df_threads = pd.DataFrame(columns=["Thread UID",
                                   "Keyword",
                                   "Reference Bank",
                                   "Fetch Datetime",
                                   "Thread Post Datetime",
                                   "Thread Author",
                                   "Thread Lastest Comment Time(Thread info)",
                                   "Thread Lastest Comment Time(Fetch inside comment)",
                                   "Thread Title",
                                   "Thread Number of Page",
                                   "Thread Theme",
                                   "Thread Likes",
                                   "Thread Dislike"])

df_comments = pd.DataFrame(columns=["Thread UID",
                                    "Comment UID",
                                    "Fetch Datetime",
                                    "Comment Floor Number",
                                    "Comment Author",
                                    "Comment Date",
                                    "Comment Time",
                                    "Comment Likes",
                                    "Comment Dislike",
                                    "Comment Replies",
                                    "Comment Context"])   



df_keyword = pd.read_excel('LIHKG_Keyword.xlsx', index_col=None)
keyword_list = df_keyword["Keyword"].values.tolist()
ref_bank_list = df_keyword["Reference Bank"].values.tolist()
df_keyword

,Keyword,Reference Bank
0,渣打信用卡,Standard Chartered
1,揸兜信用卡,Standard Chartered


In [4]:
#set driver to uc driver
driver = uc.Chrome()

#starts of the loop

for row, keyword in enumerate(keyword_list):
    ref_bank = ref_bank_list[row]
    keyword_uid = "LI-"+str(row).zfill(8)
    
    #get link
    driver.get(f"https://lihkg.com/search?q={keyword}&sort=desc_create_time&type=thread")
    time.sleep(3)
    
    #get html info
    html = driver.page_source
    soup = BeautifulSoup(html)
    
    #click left panel for scrolling
    driver.find_element(By.XPATH, '//*[@id="leftPanel"]/div[1]/ul/li[2]/a').click()

    #scroll to the end (leftpanel)
    while True:
        html = driver.page_source
        soup = BeautifulSoup(html)
        checker = soup.find_all("div",{"class":"_21IQKhlBjN2jlHS_TVgI3l"})
        try:
            if len(checker) == previous_checker:
                break
        except:
            pass
        
        previous_checker = len(checker)
        
        try:
            driver.find_element(By.CLASS_NAME, '_33r1FGqGJZF-fM1VZm7mhN').text
            time.sleep(2)
            break
        except:
            pass

        for i in range(20):
            ActionChains(driver)\
                .send_keys(" ")\
                .perform()
    
    #get latest html info
    html = driver.page_source
    soup = BeautifulSoup(html)
    
    #get thread html info
    threads = soup.find("div",{"class":"qoAmEqNpZRLf2KVKZ8DsC"})
    for thread_idx, i in enumerate(threads):
        #click thread
        click_thread(thread_idx)
        thread_uid = keyword_uid + "-" +str(thread_idx).zfill(8)
        
        #get thread info
        try:
            try:
                thread_author = i.find("span", {"class":"CxY4XDSSItTeLVg0cKCN0 A0jheqYUBHNW93KykXKEH"}).text
            except:
                thread_author = i.find("span", {"class":"CxY4XDSSItTeLVg0cKCN0 jj_3ZDzjtPixL1b2KTcpS"}).text

            thread_lastest_comment_time = i.find("span", {"class":"_37XwjAqVHtjzqzEtybpHrU"}).text
            
            ###
            #check day or hr or month function & break it when the time is not in range
            #
            #
            ###

            thread_title = i.find("span", {"class":"_20jopXBFHNQ9FUbcGHLcHH"}).text
            thread_page = i.find("div", {"class":"_26oEXjfUS_iHzbxYcZE6bD"}).text.split(" ")[0]
            thread_theme = i.find_all("a")[1].text
            thread_total_like = driver.find_elements(By.CLASS_NAME, '_8_NT40G-QNQzcSSTrRXAD')[0].get_attribute('data-score')
            thread_total_dislike = driver.find_elements(By.CLASS_NAME, '_8_NT40G-QNQzcSSTrRXAD')[1].get_attribute('data-score')

        except:

            pass
        
        #current time
        fetch_datetime = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

        #get the latest page info
        html = driver.page_source
        soup = BeautifulSoup(html)

        #loop to the end of pages
        for click in range(int(thread_page)):
            '''
            ###
            very rare case when occur a thread that have very longgggg
            context in the first post, the f5 detect will not working.
            Thus it will skip the scrolling and fetch data directly.
            ###
            '''
            try:
                find_f5 = driver.find_element(By.CLASS_NAME, '_1PhR8JHkMcET5QT2PtCA3T')
                actions = ActionChains(driver)
                actions.move_to_element(find_f5).perform()
                time.sleep(1)
            except:
                pass

        #get the latest page info
        html = driver.page_source
        soup = BeautifulSoup(html)
        
        #start of fetching comments data
        all_comment = soup.find_all("div",{"class":"_36ZEkSvpdj_igmog0nluzh"})
        for comment_idx , comment in enumerate(all_comment):
            
            comment_uid = thread_uid + "-" + str(comment_idx).zfill(8)
            
            comment_floor_number = comment.find_all("span")[0].text
            comment_author = comment.find("span",{"class":"ZZtOrmcIRcvdpnW09DzFk"}).text

            #incase the comment is folded due to not enough info
            try:
                comment_datetime = comment.find("span",{"class":"Ahi80YgykKo22njTSCzs_"})
                new_comment_datetime = str(comment_datetime).split("\"")[5]
                comment_date = new_comment_datetime.split(" ")[0]
                comment_time = new_comment_datetime.split(" ")[1]

                comment_like = comment.find_all("label")[1].text
                comment_dislike = comment.find_all("label")[3].text
            except:
                comment_datetime = ""
                comment_date = ''
                comment_time = ''
                comment_like = 0
                comment_dislike = 0

            try:
                comment_replies = comment.find_all("label")[5].text
            except:
                comment_replies = 0

            try:
                comment_context = comment.find("div",{"class":"_2cNsJna0_hV8tdMj3X6_gJ"}).text
            except:
                comment_context = ''
            
            #find earliest time
            if comment_idx == 0:
                thread_post_datetime = new_comment_datetime
            
            #get the lastest comment time
            latest_comment_time = new_comment_datetime
                
                
            #current time
            fetch_datetime = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            df_comments.loc[len(df_comments)]= [thread_uid,
                                                 comment_uid,
                                                 fetch_datetime,
                                                 comment_floor_number,
                                                 comment_author,
                                                 comment_date,
                                                 comment_time,
                                                 int(comment_like),
                                                 int(comment_dislike), 
                                                 int(comment_replies), 
                                                 comment_context]
        #save thread into to dataframe
        df_threads.loc[len(df_threads)]= [thread_uid,
                                          keyword,
                                          ref_bank,
                                          fetch_datetime,
                                          thread_post_datetime,
                                          thread_author, 
                                          thread_lastest_comment_time,
                                          latest_comment_time,
                                          thread_title,
                                          int(thread_page),
                                          thread_theme, 
                                          int(thread_total_like), 
                                          int(thread_total_dislike)]
        
        
        
        

In [5]:
df_threads

,Thread UID,Keyword,Reference Bank,Fetch Datetime,Thread Post Datetime,Thread Author,Thread Lastest Comment Time(Thread info),Thread Lastest Comment Time(Fetch inside comment),Thread Title,Thread Number of Page,Thread Theme,Thread Likes,Thread Dislike
0,LI-00000000-00000000,渣打信用卡,Standard Chartered,2023-01-02 13:26:51,2022年12月9日 09:37:13,GME散戶（暴發）,1 日前,2023年1月1日 13:20:20,突發！張渣打信用卡俾人碌左萬鳩幾蚊！！,12,財經台,680,21
1,LI-00000000-00000001,渣打信用卡,Standard Chartered,2023-01-02 13:26:55,2022年12月8日 22:23:41,一蓑煙雨,24 日前,2022年12月9日 09:32:40,渣打：近日信用卡可疑交易事件中，只有1%帳戶受影響，下星期一起陸續安排退款，並無證據顯示銀行...,2,時事台,3,18
2,LI-00000000-00000002,渣打信用卡,Standard Chartered,2023-01-02 13:26:57,2022年12月8日 22:03:18,習慣咗就冇事,24 日前,2022年12月9日 09:56:38,點解渣打ATM入錢冇得即時更新信用卡餘額,1,財經台,2,4
3,LI-00000000-00000003,渣打信用卡,Standard Chartered,2023-01-02 13:26:59,2022年12月8日 18:40:32,芝麻仁冇蘑菇,24 日前,2022年12月8日 18:40:32,渣打：客戶信用卡被盜用毋須責任 呼籲不用急於致電查詢,1,財經台,1,1
4,LI-00000000-00000004,渣打信用卡,Standard Chartered,2023-01-02 13:27:01,2022年12月7日 23:50:26,毒菇天下,24 日前,2022年12月8日 20:41:27,渣打信用卡疑被集體盜用 熱線長期唔通全城怒「虎」！ 男苦主被盜碌逾$17000報警求助！｜C...,1,財經台,3,0
5,LI-00000000-00000005,渣打信用卡,Standard Chartered,2023-01-02 13:27:02,2022年12月7日 23:39:23,藍色幽默,24 日前,2022年12月9日 12:07:17,渣打信用卡部門職員，你問我答,1,創意台,3,8
6,LI-00000000-00000006,渣打信用卡,Standard Chartered,2023-01-02 13:27:04,2022年12月7日 23:19:28,一蓑煙雨,25 日前,2022年12月7日 23:53:57,渣打信用卡疑被盜用 消委會昨日至今接4宗投訴,1,時事台,0,0
7,LI-00000000-00000007,渣打信用卡,Standard Chartered,2023-01-02 13:27:10,2022年12月7日 22:47:10,鹿特丹斯巴光年,22 日前,2022年12月8日 00:09:57,渣打信用卡客戶懷疑被大規模盜用 官方最新回應：確定非授權交易可獲退款！,4,時事台,0,21
8,LI-00000000-00000008,渣打信用卡,Standard Chartered,2023-01-02 13:27:16,2022年12月7日 22:47:10,鹿特丹斯巴光年,22 日前,2022年12月8日 00:09:57,渣打信用卡客戶懷疑被大規模盜用 官方最新回應：確定非授權交易可獲退款！,4,時事台,0,21
9,LI-00000000-00000009,渣打信用卡,Standard Chartered,2023-01-02 13:27:18,2022年12月7日 21:51:32,有冇人打波,25 日前,2022年12月7日 22:34:08,[多謝你班恐懼撚] 渣打信用卡,1,時事台,10,21


In [6]:
df_comments

,Thread UID,Comment UID,Fetch Datetime,Comment Floor Number,Comment Author,Comment Date,Comment Time,Comment Likes,Comment Dislike,Comment Replies,Comment Context
0,LI-00000000-00000000,LI-00000000-00000000-00000000,2023-01-02 13:26:51,#1,GME散戶（暴發）,2022年12月9日,09:37:13,680,21,0,\n\n\n\n\n\n覆完之後入app 睇\n我根本冇呢張尾數既卡\n做乜鳩 咩事？？
1,LI-00000000-00000000,LI-00000000-00000000-00000001,2023-01-02 13:26:51,#2,洪為民請命,2022年12月9日,09:38:01,75,308,15,坐撚咗兩日？無睇新聞？
2,LI-00000000-00000000,LI-00000000-00000000-00000002,2023-01-02 13:26:51,#3,99912345,2022年12月9日,09:39:06,83,0,0,無事 呢幾日算係正常
3,LI-00000000-00000000,LI-00000000-00000000-00000003,2023-01-02 13:26:51,#4,那日的笑容,2022年12月9日,09:55:38,77,20,10,呢鋪渣打會唔會破產清盤?
4,LI-00000000-00000000,LI-00000000-00000000-00000004,2023-01-02 13:26:51,#5,奧斯卡PS3,2022年12月9日,09:55:42,219,3,1,根本冇呢張尾數既卡
...,...,...,...,...,...,...,...,...,...,...,...
1044,LI-00000001-00000002,LI-00000001-00000002-00000020,2023-01-02 13:31:07,#21,向前走,2022年4月5日,03:41:45,3,1,2,報完之後就睇MA同銀行點樣協調同查番件事了\n\n冇做錯嘅話怕佢老母 搞大佢
1045,LI-00000001-00000002,LI-00000001-00000002-00000021,2023-01-02 13:31:07,#22,尼連住,2022年4月5日,03:41:46,159,0,3,中phishing就唔係銀行安全問題 而係樓主問題
1046,LI-00000001-00000002,LI-00000001-00000002-00000022,2023-01-02 13:31:07,#23,港豬（已醒覺）,2022年4月5日,03:42:21,15,0,1,之前俾人盜用過，短時間內碌左好多筆細數，但總額都唔多。\n\n我反而係銀行個邊發現可疑，se...
1047,LI-00000001-00000002,LI-00000001-00000002-00000023,2023-01-02 13:31:07,#24,陳年IT狗,2022年4月5日,03:48:54,51,91,2,你個case唔係盜用\n係自己授權\nsms金額係pre authorize金額\nauth...


In [7]:
#output data
df_threads.to_excel('LIHKG_Thread_Level.xlsx',encoding= 'utf-8-sig',index = None) 
df_comments.to_excel('LIHKG_Comment_Level.xlsx',encoding= 'utf-8-sig',index = None) 

C:\Users\User\anaconda3\lib\site-packages\pandas\util\_decorators.py:211: FutureWarning: the 'encoding' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'encoding'
  return func(*args, **kwargs)
